# Predicción de Rendimiento Académico usando Naive Bayes

Como se menciona en el archivo README.md, este modelo es uno de los más efectivos según varios estudios relacionados con el tema. Sin embargo, se debe probar su precisión teniendo en cuenta los datos previos del curso objetivo para determinar si el modelo es de utilidad en este caso particular.

Se comienza importando las librerías a utilizar:

In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

Para el entrenamiento del modelo, se utilizarán datos de los dos semestres anteriores al curso. En particular, serán de interés las primeras notas (AS01 y T00) para la predicción, ya que ese es el momento para detectar tempranamente aquellos casos que pueden necesitar apoyo en el desarrollo del curso.

In [2]:
# Cargado de archivos
file_2019_2 = 'data/Notas-2019-2.xlsx'
file_2020_1 = 'data/Notas-2020-1.xlsx'
df_2019_2 = pd.read_excel(file_2019_2, index_col=None, na_values=['NA'], nrows=381, usecols='B,N,AC', skiprows=2)
df_2020_1 = pd.read_excel(file_2020_1, index_col=None, na_values=['NA'], nrows=382, usecols='B,L,V,W', skiprows=1)

# Se renombran las columnas
df_2019_2.columns = ['AS01', 'T00', 'Final']
df_2020_1.columns = ['AS01', 'T00', 'Final', 'Retira']

# Se quitan quienes retiraron el curso 2020-1
df_2020_1 = df_2020_1[df_2020_1.Retira != 'RETIRO'][['AS01', 'T00', 'Final']]


Se procede a unir ambos `DataFrames` y a preprocesar los datos:

In [3]:
# Se concatenan los DataFrames y se hace el preprocesamiento
concatenate = [df_2019_2, df_2020_1]
df = pd.concat(concatenate).dropna()
df = df[df.Final != 'P'].apply(pd.to_numeric)
df.index = range(len(df.index))
df = df.apply(lambda x: x*100, axis=1)

Una vez listos los datos, se entrena el modelo y se realiza la predicción:

In [4]:
train_data = df[['AS01', 'T00', 'Final']].to_numpy()

train, test = train_test_split(train_data, test_size=0.3)

train_x, train_y  = train[:, [True, True, False]], train[:, [False, False, True]]
test_x, test_y = test[:, [True, True, False]], test[:, [False, False, True]]

train_x = train_x.astype('int')
train_y = train_y.astype('int')
test_x = test_x.astype('int')
test_y = test_y.astype('int')


gaussian = GaussianNB()
gaussian.fit(train_x, train_y)

pred_y = gaussian.predict(test_x)


Se procede a comparar los datos predichos con los reales, para saber la precisión del modelo. Es de importancia determinar con precisión quiénes aprobarán o reprobarán el curso, pero también se incluye la precisión de predicción exacta del promedio:

In [6]:
from sklearn.metrics import precision_score

precision = precision_score(bool_test, bool_pred)

precision_notas = precision_score(test_y, pred_y, average='micro')

print(f'Precisión aprobados/reprobados: {round(precision*100, 2)}%')
print(f'\nPrecisión de notas: {round(precision_notas*100, 2)}%')

Precisión aprobados/reprobados: 91.28%

Precisión de notas: 12.02%


Se obtuvo una alta precisión al determinar si un estudiante aprobará el curso o no a partir de las primeras dos evaluaciones, por lo que se establece que el modelo es de confianza para su utilización en predecir el rendimiento de quienes están actualmente en el curso.